In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.comp import ProxyFunction
from ngsolve.fem import CoefficientFunction
from netgen.occ import *
ProxyFunction.__or__ = lambda a, b: InnerProduct(a, b)
CoefficientFunction.__or__ = lambda a, b: InnerProduct(a, b)

In [ ]:

maxh = 0.05
order = 2

largeBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0.5, 0.5, 0.5))

with TaskManager(): 
    #mesh = Mesh(unit_cube.GenerateMesh(maxh=maxh ))
    mesh = Mesh(OCCGeometry(largeBrick).GenerateMesh(maxh=maxh))
h1 = H1(mesh, order = order, dirichlet = "")
hcurl = HCurl(mesh, order = order, type1 = True)

fes = hcurl*h1

(u,p) , (v,q) = fes.TnT()

Cw = Parameter(10**3)
h = specialcf.mesh_size
n = specialcf.normal(3)

def Gn(u): return Cross(n, curl(u))

def Gt(u): return Cross(n, Cross(u, n))

dS = dx(skeleton  = True, element_vb = BND, definedon=mesh.Boundaries(".*"))

In [3]:
coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

g = CF((x**2 * sin(z) * cos(y), 
        2 * z**3 * sin(x) * cos(1/3*z),
        -y**2 *-cos(3*z)*sin(x)         ))

ff = CF(GCurl(GCurl(g)) - GGrad(GDiv(g))).Compile()


In [4]:
b = BilinearForm(fes)

b += (curl(u)|curl(v)) *dx
b += (Gt(v)|Gn(u)) *dS
b += (Gt(u)|Gn(v)) *dS
b += (u|Grad(q)) *dx
b += (v|Grad(p)) *dx
b += Cw/h *(Gt(u)|Gt(v)) *dS
b += -(p|q) *ds

with TaskManager(): 
    b.Assemble()




In [5]:
#g = -pi**2*CF((1,0,0))*sin(pi*x)*sin(pi*y)*sin(pi*z)

f = LinearForm(fes)
f += (ff|v) *dx

with TaskManager():
    f.Assemble()

In [6]:
with TaskManager():
    blocks = fes.CreateSmoothingBlocks()
    prebj = b.mat.CreateBlockSmoother(blocks)



In [7]:
gf = GridFunction(fes)

from ngsolve.solvers import GMRes
with TaskManager():
    GMRes(A =b.mat,x= gf.vec, b=f.vec,pre = prebj,  printrates="\r", maxsteps = 1000) 

GMRes converged in 184 iterations to residual 3.4789613585097663e-13


In [8]:
gfu, gfp = gf.components
Draw(Norm(gfu), mesh, "gfu", clipping = {"z":-1, "function":True})

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [9]:
Draw(gfp, mesh, "gfu", clipping = {"z":-1, "function":True})

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [10]:
Draw(ff - gfu, mesh, "gfu", clipping = {"z":-1, "function":True})

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene